## Building a GPT

In [1]:
from time import time

import requests
import torch

from src.data import Data, Vocabulary
from src.gpt import GPTConfig, GPTModel
from src.utils import count_parameters, estimate_loss, get_tinyshakespeare_dataset

# Data preparation

In [3]:
text = get_tinyshakespeare_dataset()

In [4]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [5]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.


In [6]:
# here are all the unique characters that occur in this text
vocab = Vocabulary(text=text)
print(''.join(vocab.stoi.keys()))
print(vocab.size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
print(vocab.encode("hii there"))
print(vocab.decode(vocab.encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
# let's now encode the entire text dataset and store it into a torch.Tensor
data = Data(vocab.encode(text))
print(data.train_data.shape, data.train_data.dtype)

torch.Size([892315]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


# Training

In [12]:
batch_size = 32  # how many independent sequences will we process in parallel?
block_size = 8  # what is the maximum context length for predictions?
max_iters = 10001
eval_interval = 1000
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
emb_size = 32
num_heads = 4
num_layers = 3
dropout = 0.2

In [14]:
config = GPTConfig(
    vocab_size=vocab.size,
    block_size=block_size,
    emb_size=emb_size,
    num_heads=num_heads,
    num_layers=num_layers,
    dropout=dropout,
)
model = GPTModel(config=config)
print(f'models has {count_parameters(model)} parameters')

print(vocab.decode(model.generate(start_tokens=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

models has 42369 parameters
logits shape: torch.Size([4, 8, 65])
loss: 4.423056602478027

sC
NFPyHHTLetAD:RlAs-ZPEDCG'iAPnFSyHtWRci:sgdP:o:.QwZcJaGK&Na:O:tLIOvxdo:wHgaCHnTg!flJLyPQaafP
l&$xL


In [15]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [16]:
timer = time()
for steps in range(max_iters):  # increase number of steps for good results...

    # sample a batch of data
    xb, yb = data.get_batch('train', block_size=block_size, batch_size=batch_size)

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if steps % eval_interval == 0:
        losses = estimate_loss(
            eval_iters=eval_iters, model=model, data=data, block_size=block_size, batch_size=batch_size
        )
        print(
            f"step: {steps + 1}; training time: {round(time() - timer)} sec; train loss: {losses['train']:.4f}; val loss: {losses['val']:.4f}"
        )

step: 1; time: 3; train loss: 4.2451; val loss: 4.2347
step: 1001; time: 26; train loss: 1.3566; val loss: 1.3918
step: 2001; time: 49; train loss: 0.9546; val loss: 0.9705
step: 3001; time: 72; train loss: 0.7204; val loss: 0.7519
step: 4001; time: 94; train loss: 0.5869; val loss: 0.6295
step: 5001; time: 123; train loss: 0.5096; val loss: 0.5368
step: 6001; time: 160; train loss: 0.4638; val loss: 0.4970
step: 7001; time: 181; train loss: 0.4276; val loss: 0.4542
step: 8001; time: 202; train loss: 0.4079; val loss: 0.4294
step: 9001; time: 223; train loss: 0.3845; val loss: 0.4074


In [17]:
print(vocab.decode(model.generate(start_tokens=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))



Go
A
G hot heepto
I
Lovo Iil-det cofonk:
o ik sutt chot! onalt tyoule Rins chmays, we hou nenpuots?
LAsonideot morcut;
Deaystt tithatigowetarofuty ouin efrakn. ODI
Is old larkne mersoe hisees sougry, tly lef I ome!-
UME fakew illld moun at musels ce tou liend fres sult;
OU:
An, toure cners:
ot gherd uh histheat, m taty, ilod momirde che-enowthe meisml:
O, hochlangeSs caoly bmy dury.t:
I tale nos Rh peely
Ton heens yigirerempeeth ofattak, otree lor demen touke-in be thin hate were bet mit paliy 
